In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = "/content/fruit_subset_100"

import zipfile

with zipfile.ZipFile("fruit_subset_100.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/fruit_subset_100")

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)


Found 500 files belonging to 5 classes.
Using 400 files for training.
Found 500 files belonging to 5 classes.
Using 100 files for validation.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


In [6]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])


In [7]:
# Visualize 9 augmented images
import matplotlib.pyplot as plt

for images, _ in train_ds.take(1):
    augmented_images = data_augmentation(images)
    plt.figure(figsize=(10, 10))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.axis("off")
    plt.suptitle("Example Augmented Images")
    plt.show()


In [8]:
base_model = tf.keras.applications.ResNet50(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze the base


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.resnet.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


In [10]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [11]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 110s 8s/step - accuracy: 0.6991 - loss: 0.8706 - val_accuracy: 1.0000 - val_loss: 0.0154
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 137s 7s/step - accuracy: 1.0000 - loss: 0.0080 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 144s 8s/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 9.3432e-04
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 98s 8s/step - accuracy: 1.0000 - loss: 8.2860e-04 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 142s 8s/step - accuracy: 1.0000 - loss: 3.9529e-04 - val_accuracy: 1.0000 - val_loss: 7.4486e-04
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 96s 8s/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 1.0000 - val_loss: 5.6135e-04
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 1.0000 - loss: 6.2714e-04 - val_accuracy: 1.0000 - val_loss: 5.6157e-04
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 1.0000 - loss: 3.4266e

In [12]:

model.save("/content/resnet50_augmented.keras")

In [13]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


##Reflection

After training the ResNet50 model with data augmentation, the model reached 100% validation accuracy with significantly lower loss compared to the baseline. Augmentation included random flips, rotations, and zooms — which helped the model generalize better by reducing overfitting and increasing robustness to input variation.